In [ ]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms
import sys
# Add the parent directory to the path so Python can find your modules
sys.path.append(os.path.abspath('..'))
sys.path.append(os.getcwd())  # Also add current directory
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
import math
import time
from collections import defaultdict

# Global variables for tracking motion between frames
prev_keypoints = None
prev_frame_time = None
fall_history = []
alert_timestamp = None

def detect_fall(keypoints, threshold=0.3):
    """
    Enhanced fall detection algorithm based on YOLOv7-W6-Pose keypoints
    as described in the journal paper "Enhanced Fall Detection Using YOLOv7-W6-Pose for Real-Time Elderly Monitoring"
    
    Args:
        keypoints: Array of keypoints from YOLOv7-W6-Pose (shape: 17*3, where each keypoint has x, y, conf)
        threshold: Confidence threshold for keypoint detection
        
    Returns:
        tuple: (is_fall, status, conditions)
            - is_fall: Boolean indicating whether a fall is detected
            - status: String description of the current state
            - conditions: List of conditions that led to the fall detection
    """
    global prev_keypoints, prev_frame_time, fall_history, alert_timestamp
    
    # Keypoint indices as defined in YOLOv7-pose
    NOSE = 0
    L_EYE, R_EYE = 1, 2
    L_EAR, R_EAR = 3, 4
    L_SHOULDER, R_SHOULDER = 5, 6
    L_ELBOW, R_ELBOW = 7, 8
    L_WRIST, R_WRIST = 9, 10
    L_HIP, R_HIP = 11, 12
    L_KNEE, R_KNEE = 13, 14
    L_ANKLE, R_ANKLE = 15, 16
    
    try:
        # Extract keypoints and organize them for easier access
        kp = {
            'nose': keypoints[NOSE*3:(NOSE+1)*3],
            'l_eye': keypoints[L_EYE*3:(L_EYE+1)*3],
            'r_eye': keypoints[R_EYE*3:(R_EYE+1)*3],
            'l_ear': keypoints[L_EAR*3:(L_EAR+1)*3],
            'r_ear': keypoints[R_EAR*3:(R_EAR+1)*3],
            'l_shoulder': keypoints[L_SHOULDER*3:(L_SHOULDER+1)*3],
            'r_shoulder': keypoints[R_SHOULDER*3:(R_SHOULDER+1)*3],
            'l_elbow': keypoints[L_ELBOW*3:(L_ELBOW+1)*3],
            'r_elbow': keypoints[R_ELBOW*3:(R_ELBOW+1)*3],
            'l_wrist': keypoints[L_WRIST*3:(L_WRIST+1)*3],
            'r_wrist': keypoints[R_WRIST*3:(R_WRIST+1)*3],
            'l_hip': keypoints[L_HIP*3:(L_HIP+1)*3],
            'r_hip': keypoints[R_HIP*3:(R_HIP+1)*3],
            'l_knee': keypoints[L_KNEE*3:(L_KNEE+1)*3],
            'r_knee': keypoints[R_KNEE*3:(R_KNEE+1)*3],
            'l_ankle': keypoints[L_ANKLE*3:(L_ANKLE+1)*3],
            'r_ankle': keypoints[R_ANKLE*3:(R_ANKLE+1)*3]
        }
        
        # Check if keypoints are detected with sufficient confidence
        low_confidence_keypoints = []
        for k, v in kp.items():
            if v[2] < threshold:
                low_confidence_keypoints.append(k)
        
        # If more than 30% of keypoints have low confidence, return early
        if len(low_confidence_keypoints) > 0.3 * len(kp):
            return False, "low_confidence", [f"Low confidence in keypoints: {', '.join(low_confidence_keypoints)}"]
        
        # 1. Calculate the length factor based on torso length to adjust thresholds
        # as described in the journal's supplementary materials
        shoulder_midpoint = [(kp['l_shoulder'][0] + kp['r_shoulder'][0])/2, 
                            (kp['l_shoulder'][1] + kp['r_shoulder'][1])/2]
        hip_midpoint = [(kp['l_hip'][0] + kp['r_hip'][0])/2, 
                        (kp['l_hip'][1] + kp['r_hip'][1])/2]
        
        torso_length = np.sqrt((shoulder_midpoint[0] - hip_midpoint[0])**2 + 
                              (shoulder_midpoint[1] - hip_midpoint[1])**2)
        
        # If torso length is near zero (very rare), use a default value
        if torso_length < 1e-5:
            torso_length = 100  # default average torso length in pixels
        
        # 2. Calculate body dimensions and ratios
        feet_midpoint = [(kp['l_ankle'][0] + kp['r_ankle'][0])/2, 
                         (kp['l_ankle'][1] + kp['r_ankle'][1])/2]
        
        # Height of the body (vertical distance from shoulders to ankles)
        body_height = abs(shoulder_midpoint[1] - feet_midpoint[1])
        
        # Width of the body (distance between shoulders)
        body_width = abs(kp['l_shoulder'][0] - kp['r_shoulder'][0])
        
        # The conditions list will store all the criteria that suggest a fall
        conditions = []
        
        # 3. Analyze shoulder position relative to feet
        # In a normal standing posture, shoulder_y < feet_y
        # In a fall, shoulder_y approaches or exceeds feet_y
        shoulder_relative_threshold = feet_midpoint[1] - 0.5 * torso_length
        if shoulder_midpoint[1] > shoulder_relative_threshold:
            conditions.append("shoulder_near_feet")
        
        # 4. Analyze body orientation (width vs height ratio)
        # In normal standing posture, height > width
        # In a fall, especially to the side, width may exceed height
        width_height_ratio = body_width / (body_height + 1e-5)  # Adding epsilon to avoid division by zero
        if width_height_ratio > 0.8:  # Threshold from the journal
            conditions.append("horizontal_orientation")
        
        # 5. Calculate angle between torso and legs
        # As mentioned in the paper - a threshold of 45 degrees is used
        try:
            # Calculate vectors for torso and legs
            torso_vector = [hip_midpoint[0] - shoulder_midpoint[0], 
                           hip_midpoint[1] - shoulder_midpoint[1]]
            
            leg_vector = [(kp['l_knee'][0] + kp['r_knee'][0])/2 - hip_midpoint[0],
                          (kp['l_knee'][1] + kp['r_knee'][1])/2 - hip_midpoint[1]]
            
            # Calculate the angle between vectors using dot product
            dot_product = torso_vector[0]*leg_vector[0] + torso_vector[1]*leg_vector[1]
            torso_magnitude = math.sqrt(torso_vector[0]**2 + torso_vector[1]**2)
            leg_magnitude = math.sqrt(leg_vector[0]**2 + leg_vector[1]**2)
            
            if torso_magnitude > 0 and leg_magnitude > 0:
                angle_cos = dot_product / (torso_magnitude * leg_magnitude)
                angle_cos = max(-1, min(1, angle_cos))  # Ensure value is in [-1, 1] range
                angle_degrees = math.degrees(math.acos(angle_cos))
                
                # Check if angle is below threshold (45 degrees as mentioned in paper)
                if angle_degrees < 45:
                    conditions.append(f"acute_body_angle_{angle_degrees:.1f}")
        except Exception as e:
            # Skip angle calculation if there's an error
            pass
        
        # 6. Motion detection (fall speed)
        # This is crucial to distinguish between a fall and intentionally lying down
        current_time = time.time()
        
        if prev_keypoints is not None and prev_frame_time is not None:
            time_elapsed = current_time - prev_frame_time
            
            if time_elapsed > 0:
                # Calculate vertical movement speed of shoulder
                prev_shoulder_y = (prev_keypoints['l_shoulder'][1] + prev_keypoints['r_shoulder'][1]) / 2
                shoulder_y = (kp['l_shoulder'][1] + kp['r_shoulder'][1]) / 2
                shoulder_speed = (shoulder_y - prev_shoulder_y) / time_elapsed
                
                # Calculate vertical movement speed of hip
                prev_hip_y = (prev_keypoints['l_hip'][1] + prev_keypoints['r_hip'][1]) / 2
                hip_y = (kp['l_hip'][1] + kp['r_hip'][1]) / 2
                hip_speed = (hip_y - prev_hip_y) / time_elapsed
                
                # Use the maximum speed between shoulder and hip
                # High positive speed indicates downward movement (in image coordinate system)
                max_speed = max(shoulder_speed, hip_speed)
                
                # Threshold adjusted based on the literature (0.6 from your original code)
                # scaled by the torso length for body-size independence
                speed_threshold = 0.6 * (torso_length / 100)
                
                if max_speed > speed_threshold:
                    conditions.append(f"high_downward_speed_{max_speed:.1f}")
        
        # Store current keypoints and time for next frame comparison
        prev_keypoints = kp.copy()
        prev_frame_time = current_time
        
        # 7. Fall decision logic as described in the journal's algorithm
        # A fall is detected if:
        # - At least 2 conditions are met (as in your original code)
        # OR
        # - High downward speed AND at least one other condition
        is_fall = False
        if len(conditions) >= 2:
            is_fall = True
        elif any(cond.startswith("high_downward_speed") for cond in conditions) and len(conditions) >= 2:
            is_fall = True
            
        # 8. Fall history tracking for more robust detection (reduce false positives)
        # Store recent fall detections to make final decision more robust
        fall_history.append(is_fall)
        # Keep only the last 5 frames
        if len(fall_history) > 5:
            fall_history.pop(0)
            
        # Require at least 3 fall detections in last 5 frames for more robust detection
        robust_fall = sum(fall_history) >= 3
        
        # 9. Alert frequency control
        # Prevent alert spam by limiting frequency
        current_time = time.time()
        if robust_fall:
            if alert_timestamp is None or (current_time - alert_timestamp > 120):  # 2 minutes between alerts
                alert_timestamp = current_time
                return True, "fallen", conditions
            else:
                # Fall detected but alert recently sent
                return False, "alert_cooldown", conditions
                
        return False, "normal", conditions
    
    except Exception as e:
        # Reset tracking variables in case of error
        prev_keypoints = None
        prev_frame_time = None
        fall_history = []
        return False, "error", [f"Error: {str(e)}"]


class FallDetectionMetrics:
    """
    Class for tracking fall detection performance metrics
    as described in the journal's evaluation methodology.
    """
    def __init__(self):
        self.true_positives = 0   # Correctly detected falls
        self.false_positives = 0  # Incorrectly detected falls
        self.true_negatives = 0   # Correctly identified non-falls
        self.false_negatives = 0  # Missed falls
        self.results = []
        
    def update(self, is_fall_detected, is_actual_fall):
        """Update metrics based on detection results"""
        if is_fall_detected and is_actual_fall:
            self.true_positives += 1
        elif is_fall_detected and not is_actual_fall:
            self.false_positives += 1
        elif not is_fall_detected and is_actual_fall:
            self.false_negatives += 1
        else:
            self.true_negatives += 1
            
        self.results.append({
            "detected": is_fall_detected,
            "actual": is_actual_fall,
            "timestamp": time.time()
        })
    
    def calculate_metrics(self):
        """Calculate performance metrics from confusion matrix"""
        # Handle division by zero
        precision = self.true_positives / (self.true_positives + self.false_positives) if (self.true_positives + self.false_positives) > 0 else 0
        recall = self.true_positives / (self.true_positives + self.false_negatives) if (self.true_positives + self.false_negatives) > 0 else 0
        specificity = self.true_negatives / (self.true_negatives + self.false_positives) if (self.true_negatives + self.false_positives) > 0 else 0
        accuracy = (self.true_positives + self.true_negatives) / (self.true_positives + self.true_negatives + self.false_positives + self.false_negatives) if (self.true_positives + self.true_negatives + self.false_positives + self.false_negatives) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            "precision": precision,
            "recall": recall,
            "specificity": specificity,
            "accuracy": accuracy,
            "f1_score": f1_score,
            "true_positives": self.true_positives,
            "false_positives": self.false_positives,
            "true_negatives": self.true_negatives,
            "false_negatives": self.false_negatives
        }
    
    def save_results(self, filename_prefix):
        """Save detection results and metrics to files"""
        import json
        import os
        
        # Create results directory if it doesn't exist
        os.makedirs(filename_prefix, exist_ok=True)
        
        # Save raw results
        with open(os.path.join(filename_prefix, "results.json"), "w") as f:
            json.dump(self.results, f, indent=2)
        
        # Save metrics
        with open(os.path.join(filename_prefix, "metrics.json"), "w") as f:
            json.dump(self.calculate_metrics(), f, indent=2)


def load_annotations(label_dir):
    """
    Load ground truth annotations from label files
    
    Args:
        label_dir: Directory containing label files
        
    Returns:
        dict: Dictionary of annotations by video file
    """
    import os
    
    annotations = {}
    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            video_name = file.replace(".txt", ".avi")
            with open(os.path.join(label_dir, file), "r") as f:
                lines = f.readlines()
                
            # Parse annotations
            fall_frames = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 3 and parts[0] == "fall":
                    start_frame = int(parts[1])
                    end_frame = int(parts[2])
                    fall_frames.extend(list(range(start_frame, end_frame + 1)))
            
            annotations[video_name] = set(fall_frames)
    
    return annotations


def process_video(video_path, annotations, metrics, model=None):
    """
    Process a video file for fall detection
    
    Args:
        video_path: Path to the video file
        annotations: Dictionary of ground truth annotations
        metrics: FallDetectionMetrics instance for tracking performance
        model: Optional pre-loaded YOLOv7 model
    """
    import cv2
    import os
    import torch
    from torchvision import transforms
    from utils.datasets import letterbox
    from utils.general import non_max_suppression_kpt
    from utils.plots import output_to_keypoint, plot_skeleton_kpts
    
    # Initialize device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Check if model is provided, otherwise load it
    if model is None:
        # Ensure models directory exists
        os.makedirs('./models', exist_ok=True)
        
        # Check if model file exists, if not download it
        if not os.path.exists('./models/yolov7-w6-pose.pt'):
            print("Model file not found. Please download yolov7-w6-pose.pt and place it in the models directory.")
            return
            
        # Load YOLOv7-pose model
        weights = torch.load('./models/yolov7-w6-pose.pt', map_location=device, weights_only=False)
        model = weights['model'].float().eval().to(device)
        if torch.cuda.is_available():
            model = model.half()
    
    # Get video file name
    video_file = os.path.basename(video_path)
    
    # Get ground truth fall frames (if available)
    fall_frames = annotations.get(video_file, set())
    
    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        return
        
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        try:
            # Process frame for pose estimation
            # Resize and preprocess
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = letterbox(image, 960, stride=64, auto=True)[0]
            image = transforms.ToTensor()(image)
            image = torch.tensor(np.array([image.numpy()]))
            
            if torch.cuda.is_available():
                image = image.half().to(device)
            else:
                image = image.to(device)
                
            # Run model inference
            with torch.no_grad():
                output, _ = model(image)
            
            # Process output for keypoints
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'])
            
            # Check if person detected
            if len(output) > 0 and len(output[0]) > 0:
                # Get keypoints for first person detected
                keypoints = output[0][0, 7:].clone().cpu().numpy()
                
                # Check if keypoints array is valid
                if len(keypoints) > 0:
                    # Check for fall
                    is_fall_detected, status, conditions = detect_fall(keypoints)
                    
                    # Ground truth - is this frame a fall according to annotations?
                    is_actual_fall = frame_count in fall_frames
                    
                    # Update metrics
                    metrics.update(is_fall_detected, is_actual_fall)
                    
                    # Display results
                    if is_fall_detected:
                        cv2.putText(frame, f"FALL DETECTED! {status}", (10, 30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    
                    # Draw skeleton - Add error handling
                    try:
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        with_skeleton = plot_skeleton_kpts(frame, keypoints, 3)
                        frame = cv2.cvtColor(with_skeleton, cv2.COLOR_RGB2BGR)
                    except Exception as e:
                        print(f"Error plotting skeleton: {str(e)}")
                        # Continue without drawing skeleton
            
            # Display frame
            cv2.imshow('Fall Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
            frame_count += 1
            
        except Exception as e:
            print(f"Error processing frame {frame_count}: {str(e)}")
            frame_count += 1
            continue
    
    cap.release()
    cv2.destroyAllWindows()


def run_fall_detection_system(camera_index=0):
    """
    Run real-time fall detection system using webcam
    
    Args:
        camera_index: Index of camera to use
    """
    import cv2
    import torch
    import os
    from torchvision import transforms
    from utils.datasets import letterbox
    from utils.general import non_max_suppression_kpt
    from utils.plots import output_to_keypoint, plot_skeleton_kpts
    
    # Initialize device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Ensure models directory exists
    os.makedirs('./models', exist_ok=True)
    
    # Check if model file exists, if not inform user
    if not os.path.exists('./models/yolov7-w6-pose.pt'):
        print("Model file not found. Please download yolov7-w6-pose.pt and place it in the models directory.")
        return
    
    # Load YOLOv7-pose model
    weights = torch.load('./models/yolov7-w6-pose.pt', map_location=device, weights_only=False)
    model = weights['model'].float().eval().to(device)
    if torch.cuda.is_available():
        model = model.half()
    
    # Open webcam
    cap = cv2.VideoCapture(camera_index)
    
    print("Fall detection system running. Press 'q' to quit.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame for pose estimation
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = letterbox(image, 960, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        
        if torch.cuda.is_available():
            image = image.half().to(device)
        else:
            image = image.to(device)
            
        # Run model inference
        with torch.no_grad():
            output, _ = model(image)
        
        # Process output for keypoints
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'])
        
        # Check if person detected
        if len(output) > 0 and len(output[0]) > 0:
            # Get keypoints for first person detected
            keypoints = output[0][0, 7:].clone().cpu().numpy()
            
            # Check for fall
            is_fall_detected, status, conditions = detect_fall(keypoints)
            
            # Display results
            if is_fall_detected:
                cv2.putText(frame, f"FALL DETECTED! {status}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print(f"Fall detected! Conditions: {conditions}")
            else:
                cv2.putText(frame, f"Status: {status}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Draw skeleton
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            with_skeleton = plot_skeleton_kpts(frame, keypoints, 3)
            frame = cv2.cvtColor(with_skeleton, cv2.COLOR_RGB2BGR)
        else:
            cv2.putText(frame, "No person detected", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Display frame
        cv2.imshow('Fall Detection System', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


def evaluate_on_dataset(base_path, phase='test'):
    """
    Evaluate fall detection system on a dataset
    
    Args:
        base_path: Base path to dataset
        phase: Dataset phase to evaluate (train, valid, test)
    """
    import os
    import torch
    
    # Fix path issues - use proper path joining
    video_path = os.path.join(base_path, phase, 'video')
    label_path = os.path.join(base_path, phase, 'labels')
    
    # Print the actual paths to help debug
    print(f"Looking for videos in: {os.path.abspath(video_path)}")
    print(f"Looking for labels in: {os.path.abspath(label_path)}")
    
    # Check if directory exists
    if not os.path.exists(video_path):
        print(f"Error: Video directory not found at {video_path}")
        # List parent directory contents to help debugging
        parent_dir = os.path.dirname(video_path)
        if os.path.exists(parent_dir):
            print(f"Contents of {parent_dir}:")
            for item in os.listdir(parent_dir):
                print(f"  - {item}")
        return
    
    if not os.path.exists(label_path):
        print(f"Error: Labels directory not found at {label_path}")
        return
    
    # Initialize metrics
    metrics = FallDetectionMetrics()
    
    # Load model once for all videos
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Ensure models directory exists
    os.makedirs('./models', exist_ok=True)
    
    # Check if model file exists
    if not os.path.exists('./models/yolov7-w6-pose.pt'):
        print("Model file not found. Please download yolov7-w6-pose.pt and place it in the models directory.")
        return
    
    # Load YOLOv7-pose model
    print(f"Loading YOLOv7-W6-Pose model...")
    weights = torch.load('./models/yolov7-w6-pose.pt', map_location=device, weights_only=False)
    model = weights['model'].float().eval().to(device)
    if torch.cuda.is_available():
        model = model.half()
    print(f"Model loaded successfully!")
    
    # Count video files
    video_files = [f for f in os.listdir(video_path) if f.endswith(('.avi', '.mp4'))]
    print(f"\n{'='*40}")
    print(f"Processing {phase} set (Videos: {len(video_files)})")
    
    # Load annotations
    annotations = load_annotations(label_path)
    
    # Process videos
    for video_file in video_files:
        video_file_path = os.path.join(video_path, video_file)
        print(f"Processing {video_file}...")
        process_video(video_file_path, annotations, metrics, model)
    
    # Save results
    results_dir = os.path.join('./results', phase)
    os.makedirs(results_dir, exist_ok=True)
    metrics.save_results(results_dir)
    print(f"{phase} metrics saved to {results_dir}")
    
    # Print results
    res = metrics.calculate_metrics()
    print(f"\n{phase.upper():<6} Results:")
    print(f"Precision: {res['precision']:.3f}")
    print(f"Recall: {res['recall']:.3f}")
    print(f"F1 Score: {res['f1_score']:.3f}")
    print(f"Accuracy: {res['accuracy']:.3f}")
    print(f"Specificity: {res['specificity']:.3f}")
    
    return metrics


# Download the YOLOv7-W6-Pose model
def download_model():
    """
    Download the YOLOv7-W6-Pose model if it doesn't exist
    """
    import os
    import subprocess
    
    # Create models directory if it doesn't exist
    os.makedirs('./models', exist_ok=True)
    
    # Check if model already exists
    if not os.path.exists('./models/yolov7-w6-pose.pt'):
        print("Downloading YOLOv7-W6-Pose model...")
        try:
            # Try wget first
            subprocess.run(["wget", "-O", "./models/yolov7-w6-pose.pt", "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt"], check=True)
        except:
            # If wget fails, try curl
            try:
                subprocess.run(["curl", "-L", "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt", "-o", "./models/yolov7-w6-pose.pt"], check=True)
            except:
                print("Failed to download model. Please download manually from:")
                print("https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt")
                print("and place it in the ./models/ directory.")
                return False
        
        print("Download complete!")
    else:
        print("Model already exists!")
    
    return True

def evaluate_all_phases(base_path):
    """
    Evaluate fall detection system on all dataset phases
    
    Args:
        base_path: Base path to dataset
    """
    import os
    import torch
    
    # Initialize metrics for each phase
    metrics = {
        'train': FallDetectionMetrics(),
        'valid': FallDetectionMetrics(),
        'test': FallDetectionMetrics()
    }
    
    # Load model once for all videos
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Ensure models directory exists
    os.makedirs('./models', exist_ok=True)
    
    # Check if model file exists
    if not os.path.exists('./models/yolov7-w6.pt'):
        print("Model file not found. Please download yolov7-w6-pose.pt and place it in the models directory.")
        return
    
    # Load YOLOv7-pose model
    print(f"Loading YOLOv7-W6-Pose model...")
    weights = torch.load('./models/yolov7-w6-pose.pt', map_location=device, weights_only=False)
    model = weights['model'].float().eval().to(device)
    if torch.cuda.is_available():
        model = model.half()
    print(f"Model loaded successfully!")
    
    # Process all phases
    for phase in ['train', 'valid', 'test']:
        # Fix path issues - make sure path is normalized
        video_path = os.path.normpath(os.path.join(base_path, phase, 'video'))
        label_path = os.path.normpath(os.path.join(base_path, phase, 'labels'))
        
        print(f"\n{'='*40}")
        print(f"Processing {phase} set")
        print(f"Looking for videos in: {video_path}")
        
        # Check if directory exists
        if not os.path.exists(video_path):
            print(f"Error: Video directory not found at {video_path}")
            # List parent directory to help debugging
            parent_dir = os.path.dirname(video_path)
            if os.path.exists(parent_dir):
                print(f"Contents of {parent_dir}:")
                for item in os.listdir(parent_dir):
                    print(f"  - {item}")
            continue
        
        if not os.path.exists(label_path):
            print(f"Error: Labels directory not found at {label_path}")
            continue
        
        # Load annotations
        annotations = load_annotations(label_path)
        
        # Process videos
        video_files = [f for f in os.listdir(video_path) if f.endswith(('.avi', '.mp4'))]
        print(f"Found {len(video_files)} video files")
        
        for video_file in video_files:
            video_file_path = os.path.join(video_path, video_file)
            print(f"Processing {video_file}...")
            try:
                process_video(video_file_path, annotations, metrics[phase], model)
            except Exception as e:
                print(f"Error processing {video_file}: {str(e)}")
                continue
        
        # Save results
        results_dir = os.path.normpath(os.path.join('./results', phase))
        os.makedirs(results_dir, exist_ok=True)
        metrics[phase].save_results(results_dir)
        print(f"{phase} metrics saved to {results_dir}")
    
    # Print final summary
    print("\n=== FINAL RESULTS ===")
    for phase in ['train', 'valid', 'test']:
        res = metrics[phase].calculate_metrics()
        print(f"\n{phase.upper():<6} Precision: {res['precision']:.3f} | Recall: {res['recall']:.3f} | F1: {res['f1_score']:.3f}")
    
    return metrics

# For running in Jupyter notebook
# 1. Download the model first
download_success = download_model()

# 2. You can now choose what you want to do:
# Uncomment one of these lines to run:

# For webcam detection:
# run_fall_detection_system(camera_index=0)

# For dataset evaluation:
evaluate_on_dataset('./datasets/FallDataset', phase='test')

# Evaluate all phases at once
# evaluate_all_phases('../datasets/FallDataset')